## Data Cleansing

In [78]:
%matplotlib inline
import pandas as pd
import numpy as np
#from sklearn.preprocessing import LabelEncoder

In [79]:
train = pd.read_csv('train_adjusted.csv')
test = pd.read_csv('test.csv')

In [80]:
col = list(test.columns)[2:]

In [81]:
cat_feature =[each for each in col if 'ID' in each]

In [82]:
for each in col:
    if train[each].dtype=='object':
        cat_feature.append(each)

In [83]:
print(cat_feature)

['ID_metro', 'ID_railroad_station_walk', 'ID_railroad_station_avto', 'ID_big_road1', 'ID_big_road2', 'ID_railroad_terminal', 'ID_bus_terminal', 'product_type', 'sub_area', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology']


In [84]:
le = LabelEncoder()
for each in cat_feature:
    train[each] = pd.factorize(train[each], sort=True)[0]
    test[each]=pd.factorize(test[each], sort=True)[0]

In [85]:
## Data Cleansing
bad_index = train[train.life_sq > train.full_sq].index
train.ix[bad_index, "life_sq"] = np.NaN
bad_index = test[test.life_sq > test.full_sq].index
test.ix[bad_index, "life_sq"] = np.NaN
bad_index = train[train.life_sq < 5].index
train.ix[bad_index, "life_sq"] = np.NaN
bad_index = test[test.life_sq < 5].index
test.ix[bad_index, "life_sq"] = np.NaN
bad_index = train[train.full_sq < 5].index
train.ix[bad_index, "full_sq"] = np.NaN
bad_index = test[test.full_sq < 5].index
test.ix[bad_index, "full_sq"] = np.NaN
bad_index = train[train.kitch_sq >= train.life_sq].index
train.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = test[test.kitch_sq >= test.life_sq].index
test.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.kitch_sq == 0).values + (train.kitch_sq == 1).values].index
train.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = test[(test.kitch_sq == 0).values + (test.kitch_sq == 1).values].index
test.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.full_sq > 210) & (train.life_sq / train.full_sq < 0.3)].index
train.ix[bad_index, "full_sq"] = np.NaN
bad_index = test[(test.full_sq > 150) & (test.life_sq / test.full_sq < 0.3)].index
test.ix[bad_index, "full_sq"] = np.NaN
bad_index = train[train.life_sq > 300].index
train.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
bad_index = test[test.life_sq > 200].index
test.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
bad_index = train[train.build_year < 1500].index
train.ix[bad_index, "build_year"] = np.NaN
bad_index = test[test.build_year < 1500].index
test.ix[bad_index, "build_year"] = np.NaN
bad_index = train[train.build_year>10000].index
train.ix[bad_index,'build_year'] = np.NaN
bad_index = test[test.build_year>10000].index
test.ix[bad_index,'build_year'] = np.NaN
bad_index = train[train.num_room == 0].index 
train.ix[bad_index, "num_room"] = np.NaN
bad_index = test[test.num_room == 0].index 
test.ix[bad_index, "num_room"] = np.NaN
bad_index = train[(train.floor == 0).values * (train.max_floor == 0).values].index
train.ix[bad_index, ["max_floor", "floor"]] = np.NaN
bad_index = train[train.floor == 0].index
train.ix[bad_index, "floor"] = np.NaN
bad_index = train[train.max_floor == 0].index
train.ix[bad_index, "max_floor"] = np.NaN
bad_index = test[test.max_floor == 0].index
test.ix[bad_index, "max_floor"] = np.NaN
bad_index = train[train.floor > train.max_floor].index
train.ix[bad_index, "max_floor"] = np.NaN
bad_index = test[test.floor > test.max_floor].index
test.ix[bad_index, "max_floor"] = np.NaN
bad_index = train[train.state == 33].index
train.ix[bad_index, "state"] = np.NaN

In [86]:
## Export Data Set
train.to_csv('train_clean.csv',index=False)
test.to_csv('test_clean.csv',index=False)

## Define Function for feature importance

In [54]:
def get_feature_importance(model):
    Importance = model.get_fscore()
    Importance = list(Importance.items())
    Feature= []
    Score = []
    for each in Importance:
        Feature.append(each[0])
        Score.append(each[1])
    df = pd.DataFrame({'Feature':Feature,'Score':Score}).sort_values(by=['Score'],ascending=[0])
    return df  

## CV Check: Original Data

In [22]:
import xgboost as xgb
label = np.log(train['price_doc']+1)

/home/jiashen/env3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [31]:
dtrain = xgb.DMatrix(train[col],label)

In [33]:
params = {
   'eta': 0.05, ## Try 0.01,3,5
   'max_depth': 5,## Try 4,5,6
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
     verbose_eval=50, show_stdv=True,seed=42)

[0]	train-rmse:14.427+0.00330067	test-rmse:14.427+0.00742354
[50]	train-rmse:1.16222+0.000742197	test-rmse:1.16629+0.00527812
[100]	train-rmse:0.319786+0.00330906	test-rmse:0.34514+0.00782732
[150]	train-rmse:0.293503+0.00332235	test-rmse:0.330288+0.00982024
[200]	train-rmse:0.28256+0.00289879	test-rmse:0.328995+0.00967559
[250]	train-rmse:0.273673+0.00257678	test-rmse:0.328675+0.009481


In [34]:
print(len(xgb_cvalid))

245


**Do extra validation**

In [37]:
index_train = round(0.8*len(train))
training = train.iloc[:index_train,:]
testing = train.iloc[index_train:,:]
dtraining = xgb.DMatrix(training[col],np.log(training['price_doc']+1))
dtesting = xgb.DMatrix(testing[col])

In [39]:
model = xgb.train(params,dtraining,num_boost_round=245)
pred = model.predict(dtesting)

In [41]:
from sklearn.metrics import mean_squared_error as mse

In [43]:
true = np.log(testing['price_doc']+1)
print(mse(pred,true)**0.5)

0.308275258294


**Make a prediction to see the gap between CV and LB**

In [44]:
model = xgb.train(params,dtrain,num_boost_round=245)
dtest = xgb.DMatrix(test[col])
prediction = np.exp(model.predict(dtest))-1

In [46]:
sub = pd.DataFrame({'id':test['id'],'price_doc':prediction})
sub.to_csv('test_sub_original_feature.csv',index=False)

**LB Score: 0.32155**

In [55]:
get_feature_importance(model)

,Feature,Score
116,full_sq,497
54,build_year,217
193,life_sq,209
85,floor,184
213,max_floor,162
123,kitch_sq,93
50,railroad_km,85
246,num_room,70
137,metro_min_avto,70
198,state,68


## CV Check: With Additional Features

**Add Time Related Features**

In [47]:
# Add month-year
train['timestamp'] = pd.to_datetime(train['timestamp'])
month_year = (train.timestamp.dt.month + train.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
train['month_year_cnt'] = month_year.map(month_year_cnt_map)
test['timestamp'] = pd.to_datetime(test['timestamp'])
month_year = (test.timestamp.dt.month + test.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
test['month_year_cnt'] = month_year.map(month_year_cnt_map)
# Add week-year count
week_year = (train.timestamp.dt.weekofyear + train.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
train['week_year_cnt'] = week_year.map(week_year_cnt_map)
week_year = (test.timestamp.dt.weekofyear + test.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
test['week_year_cnt'] = week_year.map(week_year_cnt_map)
# Add month and day-of-week
train['month'] = train.timestamp.dt.month
train['dow'] = train.timestamp.dt.dayofweek
test['month'] = test.timestamp.dt.month
test['dow'] = test.timestamp.dt.dayofweek
# Other feature engineering
train['rel_floor'] = train['floor'] / train['max_floor'].astype(float)
train['rel_kitch_sq'] = train['kitch_sq'] / train['full_sq'].astype(float)
test['rel_floor'] = test['floor'] / test['max_floor'].astype(float)
test['rel_kitch_sq'] = test['kitch_sq'] / test['full_sq'].astype(float)
train.apartment_name=train.sub_area.astype(str) + train['metro_km_avto'].astype(str)
test.apartment_name=test.sub_area.astype(str) + train['metro_km_avto'].astype(str)
train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
test['room_size'] = test['life_sq'] / test['num_room'].astype(float)

**Add Macro Features**

In [48]:
macro_cols = ['timestamp',"balance_trade", "balance_trade_growth", "eurrub", "average_provision_of_build_contract",
"micex_rgbi_tr", "micex_cbi_tr", "deposits_rate", "mortgage_value", "mortgage_rate",
"income_per_cap", "rent_price_4+room_bus", "museum_visitis_per_100_cap", "apartment_build"]
macro = pd.read_csv('macro_c.csv')[macro_cols]

In [49]:
train = train.merge(macro,how='left',on='timestamp')
test = test.merge(macro,how='left',on='timestamp')

**Add Handmade Features**

In [50]:
def get_ratio_preschool(df):
    df['ratio_preschool'] = df['children_preschool'] / (df['children_school'] + 1)
    return df

train = get_ratio_preschool(train)
test = get_ratio_preschool(test)

In [51]:
def get_extra_area(df):
    df['extra_area'] = df['full_sq'] - df['life_sq']
    return df

train = get_extra_area(train)
test = get_extra_area(test)

In [52]:
def get_floor_ratio(df):
    df['floor_ratio'] = df['max_floor'] - df['floor']
    return df

train = get_floor_ratio(train)
test = get_floor_ratio(test)

In [53]:
def get_room_avg_size(df):
    df['room_avg_size'] = (df['life_sq'] - df['kitch_sq']) / (df['num_room'] + 1)
    return df

train = get_room_avg_size(train)
test = get_room_avg_size(test)

**Check CV**

In [56]:
col = list(test.columns)[2:]
label = np.log(train['price_doc']+1)
dtrain = xgb.DMatrix(train[col],label)

In [57]:
params = {
   'eta': 0.05, ## Try 0.01,3,5
   'max_depth': 5,## Try 4,5,6
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
     verbose_eval=50, show_stdv=True,seed=42)

[0]	train-rmse:14.427+0.00330067	test-rmse:14.427+0.00742354
[50]	train-rmse:1.16212+0.00117447	test-rmse:1.16658+0.00569655
[100]	train-rmse:0.318965+0.00357322	test-rmse:0.34543+0.00725911
[150]	train-rmse:0.29229+0.00304937	test-rmse:0.33065+0.00924185
[200]	train-rmse:0.280103+0.00233182	test-rmse:0.328999+0.00943833
[250]	train-rmse:0.269965+0.00154922	test-rmse:0.32831+0.00942992
[300]	train-rmse:0.261297+0.00189258	test-rmse:0.328122+0.00924532


In [58]:
print(len(xgb_cvalid))

310


In [59]:
model = xgb.train(params,dtrain,num_boost_round=len(xgb_cvalid))
dtest = xgb.DMatrix(test[col])
prediction = np.exp(model.predict(dtest))-1

In [62]:
sub = pd.DataFrame({'id':test['id'],'price_doc':prediction})
sub.to_csv('test_sub_additional_feature.csv',index=False)

**LB Score: 0.31944**

In [64]:
get_feature_importance(model).iloc[:25,:]

,Feature,Score
120,full_sq,572
206,life_sq,235
54,build_year,179
90,floor,170
228,max_floor,168
16,extra_area,146
180,month_year_cnt,139
40,week_year_cnt,98
17,kindergarten_km,93
141,metro_min_avto,80


## CV Check: Intersection

In [67]:
N = 30
topN_features = ['full_sq',
 'life_sq',
 'extra_area',
 'floor',
 'build_year',
 'max_floor',
 'rel_kitch_sq',
 'micex_cbi_tr',
 'eurrub',
 'kitch_sq',
 'micex_rgbi_tr',
 'kindergarten_km',
 'state',
 'room_avg_size',
 'floor_ratio',
 'hospice_morgue_km',
 'railroad_km',
 'num_room',
 'room_size',
 'balance_trade',
 'mosque_km',
 'school_km',
 'public_healthcare_km',
 'radiation_km',
 'green_zone_km',
 'workplaces_km',
 'cemetery_km',
 'theater_km',
 'build_year']

In [68]:
from sklearn import preprocessing
for f in topN_features:
    train['norm_'+f] = (train[f] - train[f].min())/(train[f].max() - train[f].min())
    test['norm_'+f] = (test[f] - test[f].min())/(test[f].max() - test[f].min())

In [69]:
import itertools
for f_1, f_2 in itertools.combinations(topN_features, 2):
    train['x_'+f_1+'_'+f_2] = train['norm_'+f_1] * train['norm_'+f_2]
    test['x_'+f_1+'_'+f_2] = test['norm_'+f_1] * test['norm_'+f_2]

In [70]:
norm_feature_name_list = ['norm_'+f for f in topN_features]
train = train.drop(norm_feature_name_list, axis=1)
test = test.drop(norm_feature_name_list, axis=1)

In [71]:
top_x_features=['x_full_sq_extra_area', 'x_full_sq_micex_cbi_tr', 'x_full_sq_tsne_2', 'x_full_sq_tsne_1', 'x_full_sq_micex_rgbi_tr', 'x_full_sq_floor', 'x_extra_area_build_year', 'x_full_sq_eurrub', 'x_full_sq_state', 'x_full_sq_num_room', 'x_full_sq_life_sq', 'x_full_sq_floor_ratio', 'x_floor_railroad_km', 'x_micex_rgbi_tr_public_healthcare_km', 'x_build_year_room_avg_size', 'x_full_sq_cemetery_km', 'x_full_sq_school_km', 'x_public_healthcare_km_workplaces_km', 'x_full_sq_kindergarten_km', 'x_school_km_cemetery_km', 'x_full_sq_kitch_sq', 'x_life_sq_floor', 'x_life_sq_school_km', 'x_full_sq_balance_trade', 'x_full_sq_max_floor', 'x_kitch_sq_cemetery_km', 'x_mosque_km_workplaces_km', 'x_full_sq_room_avg_size', 'x_max_floor_floor_ratio', 'x_green_zone_km_theater_km', 'x_micex_cbi_tr_micex_rgbi_tr', 'x_life_sq_micex_rgbi_tr', 'x_build_year_state', 'x_rel_kitch_sq_mosque_km', 'x_life_sq_kitch_sq', 'x_floor_eurrub', 'x_life_sq_micex_cbi_tr', 'x_full_sq_workplaces_km', 'x_kitch_sq_state', 'x_mosque_km_public_healthcare_km', 'x_extra_area_state', 'x_life_sq_railroad_km', 'x_kindergarten_km_theater_km', 'x_extra_area_floor', 'x_full_sq_build_year', 'x_full_sq_railroad_km', 'x_hospice_morgue_km_workplaces_km', 'x_workplaces_km_theater_km', 'x_public_healthcare_km_theater_km', 'x_mosque_km_theater_km', 'x_full_sq_public_healthcare_km', 'x_life_sq_public_healthcare_km', 'x_floor_hospice_morgue_km', 'x_extra_area_micex_rgbi_tr', 'x_max_floor_eurrub', 'x_kindergarten_km_workplaces_km', 'x_radiation_km_cemetery_km', 'x_micex_cbi_tr_workplaces_km', 'x_tsne_1_num_room', 'x_floor_num_room', 'x_state_cemetery_km', 'x_life_sq_green_zone_km', 'x_rel_kitch_sq_public_healthcare_km', 'x_micex_cbi_tr_railroad_km', 'x_rel_kitch_sq_floor_ratio', 'x_school_km_theater_km', 'x_hospice_morgue_km_mosque_km', 'x_micex_rgbi_tr_theater_km', 'x_public_healthcare_km_green_zone_km', 'x_build_year_balance_trade', 'x_full_sq_radiation_km', 'x_max_floor_tsne_1', 'x_full_sq_hospice_morgue_km', 'x_life_sq_tsne_1', 'x_floor_tsne_1', 'x_kindergarten_km_floor_ratio', 'x_kindergarten_km_mosque_km', 'x_floor_ratio_public_healthcare_km', 'x_tsne_1_railroad_km', 'x_rel_kitch_sq_state', 'x_kindergarten_km_green_zone_km', 'x_kindergarten_km_cemetery_km', 'x_public_healthcare_km_cemetery_km', 'x_life_sq_theater_km', 'x_extra_area_micex_cbi_tr', 'x_kitch_sq_tsne_2', 'x_life_sq_state', 'x_tsne_1_hospice_morgue_km', 'x_life_sq_kindergarten_km', 'x_kitch_sq_balance_trade', 'x_floor_micex_rgbi_tr', 'x_rel_kitch_sq_kindergarten_km', 'x_green_zone_km_cemetery_km', 'x_tsne_1_workplaces_km', 'x_build_year_max_floor', 'x_kindergarten_km_public_healthcare_km', 'x_state_floor_ratio', 'x_micex_rgbi_tr_green_zone_km', 'x_railroad_km_cemetery_km', 'x_life_sq_hospice_morgue_km', 'x_tsne_2_cemetery_km', 'x_micex_rgbi_tr_balance_trade', 'x_hospice_morgue_km_radiation_km', 'x_micex_cbi_tr_public_healthcare_km', 'x_balance_trade_cemetery_km', 'x_life_sq_eurrub', 'x_micex_cbi_tr_floor_ratio', 'x_micex_rgbi_tr_tsne_1', 'x_railroad_km_workplaces_km']
drop_x_feature_list = ['x_'+f_1+'_'+f_2 for f_1, f_2 in itertools.combinations(topN_features, 2) if 'x_'+f_1+'_'+f_2 not in top_x_features]
train = train.drop(drop_x_feature_list, axis=1)
test = test.drop(drop_x_feature_list, axis=1)

In [72]:
col = list(test.columns)[2:]

In [75]:
label = np.log(train['price_doc']+1)
dtrain = xgb.DMatrix(train[col],label)
dtest = xgb.DMatrix(test[col])

In [76]:
params = {
   'eta': 0.05, ## Try 0.01,3,5
   'max_depth': 5,## Try 4,5,6
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
     verbose_eval=50, show_stdv=True,seed=42)
print(len(xgb_cvalid))

[0]	train-rmse:14.427+0.00330401	test-rmse:14.427+0.00741206
[50]	train-rmse:1.16158+0.00112167	test-rmse:1.16642+0.00583089
[100]	train-rmse:0.31748+0.00371922	test-rmse:0.346475+0.00793249
[150]	train-rmse:0.289255+0.00310835	test-rmse:0.331735+0.0101384
[200]	train-rmse:0.276163+0.00249898	test-rmse:0.330123+0.00989654
[250]	train-rmse:0.265756+0.00312954	test-rmse:0.329622+0.00967272
245


In [77]:
model = xgb.train(params,dtrain,num_boost_round=len(xgb_cvalid))
prediction = np.exp(model.predict(dtest))-1
sub = pd.DataFrame({'id':test['id'],'price_doc':prediction})
sub.to_csv('test_sub_interaction_added.csv',index=False)

**LB: 0.34757**

In [87]:
get_feature_importance(model).iloc[:25,:]

,Feature,Score
249,full_sq,366
198,build_year,109
125,life_sq,94
43,month_year_cnt,87
54,floor,87
35,week_year_cnt,62
139,max_floor,57
90,metro_min_avto,48
160,park_km,48
134,x_full_sq_floor,47
